# Final Project - Beat the Bookies

## Day One - Initial Data Exploration

The goal of today is:

- Create repositories
- Add collaborators
- Add notebooks to the project
- Confirm primary data source
- Build a dataframe that includes all the relevant and desired data from our model (check google sheets for the columns). 


Using the dataset downloaded from kaggle (https://www.kaggle.com/hugomathien/soccer?) - exploring the data available for one season (2015/16) to combine the data into our own dataframe that includes the relevant information that we need for our models.

Colummns for the dataset:

- Home Team Rank
- Home Team Total Wins
- Home Team win
- Draw
- Away Team Win
- Away Team Rank

To add columns including possesion, reds, yellows, corners, shots at goal, shots on target, goal percentage, throw ins (?), fouls, free kicks. Later on when we know that we can combine the core of the table easily for all seasons.

Target:
- Home Team win y/n

### imports 

In [1]:
import pandas as pd
import numpy as np 
import sqlite3

### importing the data from sqlite

In [2]:
# create the file path (include two beatthebookies)
path = '/Users/georgebrockman/code/georgebrockman/beatthebookies/beatthebookies/data/database.sqlite'
# create the connection
conn = sqlite3.connect(path)
# create the cursor
cursor = conn.cursor()

In [3]:
# from the match table we want to select all matches played in the English Premier league for the 2015/16 season.
# Premier League - league_id = 1729
# order by stages (Game Weeks)
# need quotes around '2015/2016'
result = cursor.execute(""" SELECT * FROM "Match" m 
                        WHERE league_id = 1729 
                        AND season = '2015/2016' 
                        ORDER BY stage ASC""")

In [4]:
row = result.fetchall()
row[0] # seems to be a lot of null values for shots, s

(4389,
 1729,
 1729,
 '2015/2016',
 1,
 '2015-08-09 00:00:00',
 1987032,
 9825,
 8654,
 0,
 2,
 1,
 2,
 4,
 6,
 8,
 4,
 6,
 3,
 5,
 7,
 5,
 1,
 2,
 4,
 6,
 8,
 3,
 5,
 7,
 5,
 4,
 6,
 1,
 3,
 3,
 3,
 3,
 6,
 6,
 8,
 8,
 8,
 11,
 1,
 3,
 3,
 3,
 3,
 6,
 6,
 6,
 8,
 10,
 10,
 30859,
 26154,
 35606,
 46539,
 38521,
 159594,
 37436,
 75489,
 36378,
 196386,
 46469,
 169756,
 109897,
 35110,
 49543,
 155782,
 37169,
 575789,
 148302,
 25496,
 18506,
 192899,
 '<goal><value><comment>n</comment><stats><goals>1</goals><shoton>1</shoton></stats><event_incident_typefk>406</event_incident_typefk><coordinates><value>21</value><value>62</value></coordinates><elapsed>43</elapsed><player2>25496</player2><subtype>header</subtype><player1>148302</player1><sortorder>2</sortorder><team>8654</team><id>4659387</id><n>251</n><type>goal</type><goal_type>n</goal_type></value><value><comment>n</comment><stats><goals>1</goals><shoton>1</shoton></stats><event_incident_typefk>39</event_incident_typefk><coordinate

In [5]:
# create a DataFrame from the SQL query
matches_df = pd.read_sql(""" SELECT * FROM "Match" m 
                        WHERE league_id = 1729 
                        AND season = '2015/2016' 
                        ORDER BY stage ASC""", conn)

In [6]:
# inspect the DataFrame
matches_df.columns.values

array(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'home_player_X1',
       'home_player_X2', 'home_player_X3', 'home_player_X4',
       'home_player_X5', 'home_player_X6', 'home_player_X7',
       'home_player_X8', 'home_player_X9', 'home_player_X10',
       'home_player_X11', 'away_player_X1', 'away_player_X2',
       'away_player_X3', 'away_player_X4', 'away_player_X5',
       'away_player_X6', 'away_player_X7', 'away_player_X8',
       'away_player_X9', 'away_player_X10', 'away_player_X11',
       'home_player_Y1', 'home_player_Y2', 'home_player_Y3',
       'home_player_Y4', 'home_player_Y5', 'home_player_Y6',
       'home_player_Y7', 'home_player_Y8', 'home_player_Y9',
       'home_player_Y10', 'home_player_Y11', 'away_player_Y1',
       'away_player_Y2', 'away_player_Y3', 'away_player_Y4',
       'away_player_Y5', 'away_player_Y6', 'away_player_Y7',
       'aw

In [7]:
# we can remove all the ones not needed for home-away goal data
simple_matches = matches_df.drop(['home_player_X1',
       'home_player_X2', 'home_player_X3', 'home_player_X4',
       'home_player_X5', 'home_player_X6', 'home_player_X7',
       'home_player_X8', 'home_player_X9', 'home_player_X10',
       'home_player_X11', 'away_player_X1', 'away_player_X2',
       'away_player_X3', 'away_player_X4', 'away_player_X5',
       'away_player_X6', 'away_player_X7', 'away_player_X8',
       'away_player_X9', 'away_player_X10', 'away_player_X11',
       'home_player_Y1', 'home_player_Y2', 'home_player_Y3',
       'home_player_Y4', 'home_player_Y5', 'home_player_Y6',
       'home_player_Y7', 'home_player_Y8', 'home_player_Y9',
       'home_player_Y10', 'home_player_Y11', 'away_player_Y1',
       'away_player_Y2', 'away_player_Y3', 'away_player_Y4',
       'away_player_Y5', 'away_player_Y6', 'away_player_Y7',
       'away_player_Y8', 'away_player_Y9', 'away_player_Y10',
       'away_player_Y11', 'home_player_1', 'home_player_2',
       'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6',
       'home_player_7', 'home_player_8', 'home_player_9',
       'home_player_10', 'home_player_11', 'away_player_1',
       'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5',
       'away_player_6', 'away_player_7', 'away_player_8', 'away_player_9',
       'away_player_10', 'away_player_11', 'goal', 'shoton', 'shotoff',
       'foulcommit', 'card', 'cross', 'corner', 'possession', 'B365H',
       'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH',
       'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'SJH',
       'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH',
       'BSD', 'BSA'], axis=1)

In [8]:
simple_matches.head(5)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,4389,1729,1729,2015/2016,1,2015-08-09 00:00:00,1987032,9825,8654,0,2
1,4390,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987033,8678,10252,0,1
2,4391,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987034,8455,10003,2,2
3,4392,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987035,8668,9817,2,2
4,4393,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987036,8197,8472,4,2


In [9]:
simple_matches['home_goals']= simple_matches.groupby('home_team_api_id')['home_team_goal'].cumsum().fillna(0)

In [10]:
simple_matches['away_goals']= simple_matches.groupby('away_team_api_id')['away_team_goal'].cumsum().fillna(0)

In [11]:
simple_matches

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_goals,away_goals
0,4389,1729,1729,2015/2016,1,2015-08-09 00:00:00,1987032,9825,8654,0,2,0,2
1,4390,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987033,8678,10252,0,1,0,1
2,4391,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987034,8455,10003,2,2,2,2
3,4392,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987035,8668,9817,2,2,2,2
4,4393,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987036,8197,8472,4,2,4,2
5,4394,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987037,10260,8586,1,0,1,0
6,4395,1729,1729,2015/2016,1,2015-08-09 00:00:00,1987038,10261,8466,2,2,2,2
7,4396,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987039,9850,9826,1,3,1,3
8,4397,1729,1729,2015/2016,1,2015-08-09 00:00:00,1987040,10194,8650,0,1,0,1
9,4398,1729,1729,2015/2016,1,2015-08-10 00:00:00,1987041,8659,8456,0,3,0,3


In [12]:
simple_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 13 columns):
id                  380 non-null int64
country_id          380 non-null int64
league_id           380 non-null int64
season              380 non-null object
stage               380 non-null int64
date                380 non-null object
match_api_id        380 non-null int64
home_team_api_id    380 non-null int64
away_team_api_id    380 non-null int64
home_team_goal      380 non-null int64
away_team_goal      380 non-null int64
home_goals          380 non-null int64
away_goals          380 non-null int64
dtypes: int64(11), object(2)
memory usage: 38.7+ KB


In [13]:
simple_matches['home_wins'] = simple_matches.home_team_goal > simple_matches.away_team_goal
simple_matches['draws'] = simple_matches.home_team_goal == simple_matches.away_team_goal
simple_matches['away_wins'] = simple_matches.home_team_goal < simple_matches.away_team_goal
simple_matches

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_goals,away_goals,home_wins,draws,away_wins
0,4389,1729,1729,2015/2016,1,2015-08-09 00:00:00,1987032,9825,8654,0,2,0,2,False,False,True
1,4390,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987033,8678,10252,0,1,0,1,False,False,True
2,4391,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987034,8455,10003,2,2,2,2,False,True,False
3,4392,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987035,8668,9817,2,2,2,2,False,True,False
4,4393,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987036,8197,8472,4,2,4,2,True,False,False
5,4394,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987037,10260,8586,1,0,1,0,True,False,False
6,4395,1729,1729,2015/2016,1,2015-08-09 00:00:00,1987038,10261,8466,2,2,2,2,False,True,False
7,4396,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987039,9850,9826,1,3,1,3,False,False,True
8,4397,1729,1729,2015/2016,1,2015-08-09 00:00:00,1987040,10194,8650,0,1,0,1,False,False,True
9,4398,1729,1729,2015/2016,1,2015-08-10 00:00:00,1987041,8659,8456,0,3,0,3,False,False,True


In [14]:
# convert booleans into binary numbers 
simple_matches['home_wins'] = simple_matches['home_wins'].astype(int)
simple_matches['draws'] = simple_matches['draws'].astype(int)
simple_matches['away_wins'] = simple_matches['away_wins'].astype(int)


In [15]:
simple_matches.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_goals,away_goals,home_wins,draws,away_wins
0,4389,1729,1729,2015/2016,1,2015-08-09 00:00:00,1987032,9825,8654,0,2,0,2,0,0,1
1,4390,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987033,8678,10252,0,1,0,1,0,0,1
2,4391,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987034,8455,10003,2,2,2,2,0,1,0
3,4392,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987035,8668,9817,2,2,2,2,0,1,0
4,4393,1729,1729,2015/2016,1,2015-08-08 00:00:00,1987036,8197,8472,4,2,4,2,1,0,0


In [25]:
# create cumulative home, draw and away wins grouping by each team id

simple_matches['cumulative_home_wins_home'] = simple_matches.groupby('home_team_api_id')['home_wins'].cumsum().fillna(0)
simple_matches['cumulative_away_wins_home'] = simple_matches.groupby('home_team_api_id')['home_wins'].cumsum().fillna(0)
simple_matches['cumulative_home_wins_away'] = simple_matches.groupby('away_team_api_id')['away_wins'].cumsum().fillna(0)
# simple_matches['cumulative_draws'] = simple_matches.groupby('home_team_api_id')['draws'].cumsum().fillna(0)
simple_matches['cumulative_away_wins_away'] = simple_matches.groupby('away_team_api_id')['away_wins'].cumsum().fillna(0)


In [26]:
simple_matches.tail(20)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,away_goals,home_wins,draws,away_wins,cumulative_home_wins,cumulative_away_wins,cumulative_home_wins_home,cumulative_home_wins_away,cumulative_away_wins_away,cumulative_away_wins_home
360,4689,1729,1729,2015/2016,37,2016-05-07 00:00:00,1989070,10252,10261,0,...,12,0,1,0,2,2,2,2,2,2
361,4690,1729,1729,2015/2016,37,2016-05-07 00:00:00,1989071,8678,8659,1,...,14,0,1,0,5,4,5,4,4,5
362,4691,1729,1729,2015/2016,37,2016-05-07 00:00:00,1989072,9826,10194,2,...,19,1,0,0,6,6,6,6,6,6
363,4692,1729,1729,2015/2016,37,2016-05-07 00:00:00,1989073,8197,8668,3,...,24,1,0,0,12,5,12,5,5,12
364,4693,1729,1729,2015/2016,37,2016-05-08 00:00:00,1989074,8650,9817,2,...,20,1,0,0,8,6,8,6,6,8
365,4694,1729,1729,2015/2016,37,2016-05-08 00:00:00,1989075,8456,9825,2,...,34,0,1,0,12,8,12,8,8,12
366,4695,1729,1729,2015/2016,37,2016-05-07 00:00:00,1989076,9850,10260,0,...,22,0,0,1,6,7,6,7,7,6
367,4696,1729,1729,2015/2016,37,2016-05-07 00:00:00,1989077,8472,8455,3,...,27,1,0,0,6,7,6,7,7,6
368,4697,1729,1729,2015/2016,37,2016-05-08 00:00:00,1989078,8586,8466,1,...,20,0,0,1,10,7,10,7,7,10
369,4698,1729,1729,2015/2016,37,2016-05-07 00:00:00,1989079,8654,10003,1,...,22,0,0,1,9,4,9,4,4,9


In [18]:
# cumulative home team wins per team 


## Day two - Advanced Exploration (multiple seasons)